In [1]:
import pyarrow
import pyarrow.parquet as pq
import pandas as pd
import gzip
import ast
from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")

Extraigo la informacion de los 3 archivos .json.gz, descomprimiendolos y en algunos casos paso de texto a diccionario las lineas de los archivos, para asi poder leerlos

In [37]:
with gzip.open(r"datasets\steam_games.json.gz", "rt") as f:
    df_games = pd.read_json(f, lines=True)

In [3]:
filas = []
with gzip.open(r"datasets\user_reviews.json.gz", "rt", encoding="utf-8") as f:
    for line in f.readlines():
        filas.append(ast.literal_eval(line))
    df_reviews = pd.DataFrame(filas)

In [2]:
filas1 = []
with gzip.open(r"datasets\users_items.json.gz", "rt", encoding="utf-8") as f:
    for line in f.readlines():
        filas1.append(ast.literal_eval(line))
    df_items = pd.DataFrame(filas1)

Visualizo el dataframe

In [ ]:
df_games

Elimino nulos y quito columnas innecesarias del dataframe

In [38]:
df_games = df_games.dropna()
df_games = df_games.drop(columns=["title", "url", "reviews_url"])
df_games

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich


Transformo las fechas con formato irregular a fechas legibles, a su vez elimino los registros en donde aparecen "SOON" o "SOON™", luego extraigo el año de las fecha y lo transformo a tipo int para poder manipularlo con mayor facilidad y elimino nulos nuevamente

In [39]:
df_games = df_games[~df_games["release_date"].isin(["SOON", "SOON™"])]
def format_date(x):
    if pd.notna(x):
        date = pd.to_datetime(f"01 {x}", format="%d %b %Y", errors="coerce")
        if not pd.isna(date):
            return date.strftime("%Y-%m-%d")
    return x

df_games["release_date"] = df_games["release_date"].apply(format_date)
df_games["release_date"] = pd.to_datetime(df_games["release_date"], format="%Y-%m-%d", errors="coerce")
df_games["release_date"] = df_games["release_date"].dt.year
df_games = df_games.dropna()
df_games["release_date"] = df_games["release_date"].astype(int)


Creo una funcion para convertir a tipo float la columna price y en caso de no ser posible debido a la presencia de datos como "Free to Play", entonces estos pasaran a ser "0", tambien paso a tipo int la columna id

In [40]:
def pasar_a_float(valor):
    try:
        return float(valor)
    except (ValueError, TypeError):
        return 0
    
df_games["price"] = df_games["price"].apply(pasar_a_float)
df_games["id"] = df_games["id"].astype(int)

Elimino duplicados en los nombres de los juegos

In [41]:
df_games = df_games.drop_duplicates(subset=["app_name"])

Tomo las columnas que me van a ser de utilidad para las funciones de la API, filtro el dataframe y asigno el resultado a "data_games" y luego lo paso a CSV como "df_games"

In [45]:
columnas = ["genres", "app_name", "release_date", "price", "early_access", "id", "developer"]
data_games = df_games[columnas]
data_games.to_csv("post_ETL/df_games.csv", index=False)

Visualizo el dataframe

In [ ]:
df_reviews

Elimino columna innecesaria

In [10]:
df_reviews = df_reviews.drop(columns="user_url")


Realizo el desanidado de la columna "reviews" y asigno el resultado a "df_desanidado_review"

In [11]:
df_reviews_filtered = df_reviews.dropna(subset=["reviews"])
df_review_desanidado = pd.DataFrame(df_reviews_filtered["reviews"])
desanidado_re = []
for index, row in df_review_desanidado.iterrows():
    lista_review = row["reviews"]
    if lista_review:
        df_review = pd.json_normalize(lista_review)
        df_review["row_index"] = index
        desanidado_re.append(df_review)

df_desanidado_review = pd.concat(desanidado_re, ignore_index=True)

Elimino la columna reviews del dataframe "df_reviews" ya que esta columna ahora esta en el dataframe "df_desanidado_review"

In [12]:
df_reviews = df_reviews.drop(columns="reviews")

Visualizo el dataframe desanidado

In [13]:
df_desanidado_review

,funny,posted,last_edited,item_id,helpful,recommend,review,row_index
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1
...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,25797
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,25797
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,25798
59303,,Posted July 20.,,730,No ratings yet,True,:D,25798


Realizo un analisis de sentimiento de los comentarios hechos por los usuarios en las recomendaciones, asignando 0 en caso de ser un comentario negativo, 1 si es neutro y 2 si es positivo, en caso de no haber comentario se considerara neutro es decir 1

In [14]:
def analizar_sentimiento(texto):
  if texto.strip() == "":
    sentimiento = 1
    return sentimiento
  else:
    blob = TextBlob(texto)
    polaridad = blob.sentiment.polarity
    if polaridad > 0.2:
      sentimiento = 2
    elif polaridad < -0.2:
      sentimiento = 0
    else:
      sentimiento = 1
  return sentimiento

df_desanidado_review["sentiment_analysis"] = df_desanidado_review["review"].apply(analizar_sentimiento)

Filtro el dataframe a las columnas que voy a necesitar para las funciones de la API y realizo un merge para unir las reviews desanidadas y el dataframe original de reviews, elimino la columna index y luego lo paso a CSV con el nombre "df_reviews_desanidado"

In [15]:
columnas_re = ["item_id", "recommend", "row_index", "sentiment_analysis"]
data_reviews1 = df_desanidado_review[columnas_re]
data_reviews = df_reviews.reset_index().merge(data_reviews1, left_on="index", right_on="row_index", how="inner")
data_reviews = data_reviews.drop(columns="index")
data_reviews.to_csv("post_ETL/df_reviews_desanidado.csv", index=False)

Visualizo el dataframe resultante del merge

In [34]:
data_reviews

,user_id,item_id,recommend,row_index,sentiment_analysis
0,76561197970982479,1250,True,0,1
1,76561197970982479,22200,True,0,2
2,76561197970982479,43110,True,0,1
3,js41637,251610,True,1,1
4,js41637,227300,True,1,1
...,...,...,...,...,...
59300,76561198312638244,70,True,25797,2
59301,76561198312638244,362890,True,25797,1
59302,LydiaMorley,273110,True,25798,1
59303,LydiaMorley,730,True,25798,2


Visualizo el ultimo dataframe a tratar

In [3]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Elimino columnas innecesarias

In [4]:
df_items = df_items.drop(columns=["steam_id", "user_url"])

Elimino usuarios duplicados y visualizo

In [5]:
df_items = df_items.drop_duplicates(subset=["user_id"])
df_items

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...
88305,76561198323066619,22,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,[]
88308,76561198329548331,7,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Realizo el desanidado de la columna "items" y guardo el resultado en "df_desanidado_items"

In [6]:
df_items_filtered = df_items.dropna(subset=["items"])
df_items_desanidado = pd.DataFrame(df_items_filtered["items"])
desanidado_it = []
for index, row in df_items_desanidado.iterrows():
    lista_items = row["items"]
    if lista_items:
        df_item = pd.json_normalize(lista_items)
        
        df_item["row_index"] = index
        desanidado_it.append(df_item)

df_desanidado_items = pd.concat(desanidado_it, ignore_index=True)

Elimino la columna "items" del dataframe original y luego lo paso a CSV como "df_items"

In [9]:
df_items = df_items.drop(columns="items")
df_items.to_csv("post_ETL/df_items.csv", index=False)

En el dataframe desanidado cambio la columna "item_id" a tipo int para volverlo mas manipulable y elimino columnas innecesarias para las funciones de la API

In [7]:
df_desanidado_items["item_id"] = df_desanidado_items["item_id"].astype(int)
df_desanidado_items = df_desanidado_items.drop(columns="playtime_2weeks")
df_desanidado_items = df_desanidado_items.drop(columns="item_name")

Visualizo el resultado

In [9]:
df_desanidado_items

,item_id,item_name,playtime_forever,row_index
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
5094077,346330,BrainBread 2,0,88308
5094078,373330,All Is Dust,0,88308
5094079,388490,One Way To Die: Steam Edition,3,88308
5094080,521570,You Have 10 Seconds 2,4,88308


Almaceno el resultado en un archivo tipo "parquet" debido a la magnitud de este dataframe, gracias al control de duplicados, a la eliminacion de columnas innecesarias y a la utilizacion de parquet con el metodo de compresion "SNAPPY", consegui que pasara de pesar casi 200 megas (como CSV) a pesar 18 megas (como Parquet).

In [13]:
compression = "SNAPPY"
pq.write_table(table=pyarrow.Table.from_pandas(df_desanidado_items), where="post_ETL/df_items_desanidado.parquet", compression=compression)